Script for creating visualisation

In [34]:
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import pandas as pd

In [2]:
# load geometry data
vic_geo_df = gpd.read_file('../data/VIC_LOCALITY_POLYGON_shp.geojson')

In [5]:
# load active cases data
df_all_dates = pd.read_csv('../data/all_dates.csv')

In [6]:
vic_geo_df = pd.merge(vic_geo_df[['VIC_LGA__3', 'geometry']], df_all_dates, how = 'inner', left_on = 'VIC_LGA__3', right_on = 'lga', sort = True)

In [9]:
cmap = LinearSegmentedColormap.from_list(
    'covid_custom', [(0,'white'), (0.01, '#fffa86'), (0.25, 'yellow'), (0.5, 'red'), (0.75, 'brown'), (1, 'black')])

start by mapping just the melbourne lgas

In [10]:
melb_lgas = ['Banyule','Bayside','Boroondara','Brimbank','Cardinia','Casey','Darebin','Frankston','Glen Eira',
'Greater Dandenong','Hobsons Bay','Hume','Kingston','Knox','Manningham','Maribyrnong','Maroondah','Melbourne',
'Melton','Monash','Moonee Valley','Moreland','Mornington Peninsula','Nillumbik','Port Phillip','Stonnington',
'Whitehorse','Whittlesea','Wyndham','Yarra','Yarra Ranges']

melb_lgas = [lga.upper() for lga in melb_lgas]

In [13]:
melb_geo_df = vic_geo_df[vic_geo_df['lga'].isin(melb_lgas)]

In [22]:
date_list = vic_geo_df.columns.tolist()

date_list.remove('VIC_LGA__3')
date_list.remove('geometry')
date_list.remove('lga')

In [42]:
for i in range(len(date_list)):
    covid_map = melb_geo_df.plot(figsize = (14,10), column = date_list[i], cmap = cmap, k = 150, vmin = 0, vmax = 1000, 
                 edgecolor = 'lightblue', legend = True)
    covid_map.axis('off')
    plt.savefig('../data/images/melb/total_cases/' + str(i).zfill(3) + '.png')
    plt.close()